In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
from datetime import datetime, timedelta
from typing import Any, List

import numpy as np
import pandas as pd
import pandera as pa
import pytz
from ib_insync import (
    IB,
    Contract,
    FuturesOption,
    Index,
    Ticker,
    util,
)
from loguru import logger
from loman import ComputationFactory, input_node, calc_node
from tqdm import tqdm

from ib_insync_options.etl.ibkr_etl import map_ticker_to_exchange, MarketDataType
from ib_insync_options.utils.date_utils import calc_effective_date_of_dt
from ib_insync_options.utils.dict_utils import gen_json
from ib_insync_options.utils.formatting_utils import lower_camel_case_to_snake_case
from ib_insync_options.utils.list_utils import chunks
from ib_insync_options.utils.networking import get_ibkr_host_ip
from ib_insync_options.etl.facets import FacetCoreDfColumns, CoreFacetSchema
from ib_insync_options.etl.ibkr_etl import IbkrFutureOptionData, IbkrDownloader

import pytz
from ib_insync_options.utils.date_utils import calc_effective_date_of_dt
from ib_insync_options.etl.instrument_multiplexer import InstrumentMultiplexer

In [ ]:
# required for jupyter notebook
util.startLoop()
ib = IB()

host = get_ibkr_host_ip()
ib.connect(host, 7496, clientId=3, timeout=30)

In [ ]:
def save(gd: IbkrFutureOptionData, df: pd.DataFrame):
    """
    Save gd results/df to the ibkr cache.

    ```
    save(gd, gd.v.enrich_option_tickers_df_with_price)
    ```

    Args:
        gd (IbkrFutureOptionData): _description_
        df (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    df = df[df.edt.notnull()].copy()

    utc_now = datetime.now(pytz.utc)
    mt_now = utc_now.astimezone(pytz.timezone("US/Mountain"))
    data_effective_date = calc_effective_date_of_dt(mt_now)
    # data_effective_date_str: str = data_effective_date.strftime("%Y-%m-%d")

    assert len(df.symbol.unique()) == 1
    ticker = df.symbol.unique()[0]

    # NOTE
    # _ibkr_cache_write - writes a table called ibkr_option_core_facet, in a database called option_pricer_dev
    # f"postgresql+psycopg2://postgres:@localhost:5432/option_pricer_dev"
    # if you need to change the db connection string, see finx_optionpricer/etl/db_ops.py, gen_engine()

    return InstrumentMultiplexer._ibkr_cache_write(
        asset_class=gd.v.asset_category,
        ticker=ticker,
        effective_date=data_effective_date,
        df=df,
    )


def command_execute_graph_download(
    ib: IB,
    market_data_type: MarketDataType = MarketDataType.LIVE,
    symbol: str = "GC",
    dte_max: int = 30,
    pre_fetch_option_filters: List[dict] = None,
):
    """
    Download futures option data for a given underlying symbol.

    Steps:
    1. initialize the graph data object
    2. initialize the underlying symbol and pre-fetch filters
    3. initialize the ibkr downloader
    4. fetch the option chain for the underlying
    5. compute the option chain df
    6. fetch the underlying df
    7. enrich the option chain df with the underlying tickers
    8. apply fetch filters
    9. generate qualified futures options
    10. fetch tickers for qualified futures options
    11. generate option tickers df
    12. enrich option tickers df with the underlying price (because IBKR will drop the underlying price)

    Example,
    ```
    pre_fetch_option_filters_gc = [
        {"moneyness_gte": 95.0},
        {"moneyness_lte": 105.0},
        {"strike_modulus_eq": 10.0},
    ]
    gd = command_execute_graph_download(
        ib=ib,
        symbol="GC",
        dte_max=30,
        pre_fetch_option_filters=pre_fetch_option_filters_gc
    )
    save(gd, gd.v.enrich_option_tickers_df_with_price)
    ```

    Args:
        ib (IB): ib_insync IB object
        symbol (str, optional): underlying symbol. Defaults to "GC".
        dte_max (int, optional): max DTE. Defaults to 30.
        pre_fetch_option_filters (List[dict], optional): pre-fetch option filters. Defaults to None.
        existing_graph_data (IbkrFutureOptionData, optional): existing graph data object. Defaults to None.

    Returns:
        IbkrFutureOptionData: graph data object
    """
    # graph data object
    gd = IbkrFutureOptionData()

    # initialize the underlying symbol and pre-fetch filters
    gd.add_node("underlying_symbol", value=symbol)
    gd.add_node(
        name="pre_fetch_option_filters",
        value=pre_fetch_option_filters,
    )
    gd.add_node("dte_max", value=dte_max)
    gd.compute_all()

    # initialize the ibkr downloader
    ibkr_downloader = IbkrDownloader(ib, market_data_type=market_data_type)

    # fetch the option chain for the underlying
    raw_option_chain = ibkr_downloader.fetch_option_chain_for_underlying(symbol)
    gd.add_node("raw_option_chain", value=raw_option_chain)
    gd.compute("option_chain_df")

    # compute the option chain df
    gd.compute_all()

    xdf = gd.v.filter_option_chain_by_max_expiry_date
    underlying_df = ibkr_downloader.fetch_underlying_df(option_chain_df=xdf)
    gd.add_node("underlying_df", value=underlying_df)
    gd.compute("underlying_df_calc_price")
    gd.compute("enrich_option_chain_df_with_underlying_tickers")

    # apply fetch filters so only get the desired futures options
    gd.compute("apply_fetch_filters")

    qualified_futures_options = ibkr_downloader.gen_qualified_futures_options(
        option_chain_df=gd.v.apply_fetch_filters
    )
    gd.add_node("qualified_futures_options", value=qualified_futures_options)

    futures_option_tickers = ibkr_downloader.fetch_tickers_for_contracts(
        qualified_futures_options
    )
    gd.add_node("futures_option_tickers", value=futures_option_tickers)
    df = ibkr_downloader.gen_option_tickers_df(tickers=futures_option_tickers)
    gd.add_node("option_tickers_df", value=df)

    gd.compute("gen_qualified_fops_df")

    gd.compute("enrich_option_tickers_df_with_price")

    return gd

In [ ]:
fetch_option_filters_es = [
    {"moneyness_gte": 98.0},
    {"moneyness_lte": 102.0},
    {"strike_modulus_eq": 50.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="ES",
    dte_max=21,
    pre_fetch_option_filters=fetch_option_filters_es,
    # use frozen market data when market is closed
    market_data_type=MarketDataType.FROZEN,
    # use live market data when market is open
    # market_data_type=MarketDataType.LIVE
)

save(gd, gd.v.enrich_option_tickers_df_with_price)

In [ ]:
pre_fetch_option_filters_cl = [
    {"moneyness_gte": 95.0},
    {"moneyness_lte": 105.0},
    {"strike_modulus_eq": 5.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="CL",
    dte_max=90,
    pre_fetch_option_filters=pre_fetch_option_filters_cl,
    market_data_type=MarketDataType.FROZEN,
)
save(gd, gd.v.enrich_option_tickers_df_with_price)

In [ ]:
pre_fetch_option_filters_gc = [
    {"moneyness_gte": 95.0},
    {"moneyness_lte": 105.0},
    {"strike_modulus_eq": 10.0},
]
gd = command_execute_graph_download(
    ib=ib,
    symbol="GC",
    dte_max=90,
    pre_fetch_option_filters=pre_fetch_option_filters_gc,
    market_data_type=MarketDataType.FROZEN,
)
save(gd, gd.v.enrich_option_tickers_df_with_price)